In [1]:
from scipy.spatial import ckdtree
import numpy as np
from tqdm import tqdm_notebook as tqdm
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import ipywidgets
from ipywidgets import interact, IntSlider
from IPython.display import display

In [2]:
l = 1
d = l*1e-2
epsilon = d*1e-3

In [3]:
N = 200 #number of vertexes
Frames = 10 #number of frames between adding vercites
M = 20 #number of segment adding iteration between adding vertices
M = int(M/Frames)

In [14]:
def random_vertex():
    x, y = np.random.random(2)*2*l-l
    return np.array([x,y])

def find_segments(vertices, segments):
    active_segments = [] #segmenty które będą się rozrastać
    segments_vertices = [] #krawędzie do których będzie rozrastać się segment na odpowiednim miejscu powyżej
    tree = ckdtree.cKDTree(segments)
    for i in range(len(vertices)-1,-1,-1):
        dist, nearest_segments = tree.query(vertices[i])
        if dist < d:
            del vertices[i]
    for i in range(len(vertices)):
        dist, nearest_segments = tree.query(vertices[i],5)
        close = [True]
        close.extend((np.diff(dist)<epsilon).tolist())
        for s in nearest_segments[close]:
            active_segments.append(segments[s])
            segments_vertices.append([i])
            
#             if s in active_segments:
# #                 index = active_segments.index(s)
# #                 segments_vertices[index].extend([i])
#                 active_segments.append(s)
#                 segments_vertices.append([i])
#             else:
#                 active_segments.append(s)
#                 segments_vertices.append([i])
    
    return active_segments, segments_vertices

def segment_adding(M:int,active_vertices,active_segments,segments_vertices,segments,visual):
    for m in range(M):
        for i in range(len(active_segments)-1, -1, -1):
            del_x = active_vertices[segments_vertices[i][0]][0] - active_segments[i][0]
            del_y = active_vertices[segments_vertices[i][0]][1] - active_segments[i][1]
            r = (del_x**2 + del_y**2)**0.5
            if r > d:
                new_seg = (active_segments[i][0]+del_x*d/r,active_segments[i][1]+del_y*d/r)
                segments.append(new_seg)
                visual.append([active_segments[i],new_seg])
                active_segments[i] = new_seg
            else:
                del segments_vertices[i]
                del active_segments[i]
#                         segments_vertices.remove(segments_vertices[i])
#                         active_segments.remove(active_segments[i])

In [6]:
def animated_frames(animation_segments, animation_vertices):
    
    plt.close()
    fig, ax = plt.subplots()
    title = ax.set_title("Iteration 0")
    def update(f: int = 0):
        ax.cla()
        frame_segments = np.swapaxes(animation_segments[f],0,2)
#         frame_segments = animation_segments[f]
        frame_vertices = animation_vertices[f]
        if frame_vertices.size > 0:
            scatterplot = ax.scatter(frame_vertices[:,0], frame_vertices[:,1])
        # TODO optimize the line below - done?
        lines = ax.plot(frame_segments[0], frame_segments[1])
#         lines = [ax.plot(segment[:, 0], segment[:, 1]) for segment in frame_segments]
        
        title = ax.set_title(f"Iteration {f}/{len(animation_segments)-1}")
        fig.canvas.draw()
    plt.show()

    play = ipywidgets.Play(
        interval=500,
        value=0, 
        min=0,
        max=len(animation_segments)-1,
        step=1,
        description="Press play",
        disabled=False
        )
    
    slider = IntSlider(min=0, max=len(animation_segments)-1, step=1)
#     w = interactive()
    link = ipywidgets.jslink((play, 'value'), (slider, 'value'))
    display(slider)
#     ipywidgets.HBox([play, slider])
    # 1.png
    return interact(update, f=play);

In [15]:
def run(N=N,M=M,visualization=None):
    segments = [(0,0)]
    active_vertices = []
    visual = []
    animation_vertices = []
    animation_segments = []
    for n in tqdm(range(N)):
        active_vertices.append(random_vertex())
        active_segments, segments_vertices = find_segments(active_vertices, segments)
        for f in range(Frames):
            segment_adding(M,active_vertices,active_segments,segments_vertices,segments,visual)
#             for m in range(M):
#                 for i in range(len(active_segments)-1, -1, -1):
#                     del_x = active_vertices[segments_vertices[i][0]][0] - active_segments[i][0]
#                     del_y = active_vertices[segments_vertices[i][0]][1] - active_segments[i][1]
#                     r = (del_x**2 + del_y**2)**0.5
#                     if r > d:
#                         new_seg = (active_segments[i][0]+del_x*d/r,active_segments[i][1]+del_y*d/r)
#                         segments.append(new_seg)
#                         visual.append([active_segments[i],new_seg])
#                         active_segments[i] = new_seg
#                     else:
#                         del segments_vertices[i]
#                         del active_segments[i]
# #                         segments_vertices.remove(segments_vertices[i])
# #                         active_segments.remove(active_segments[i])
            if len(active_segments) != 0:
                animation_vertices.append(np.array(active_vertices.copy()))
                animation_segments.append(np.array(visual.copy()))

    active_segments, segments_vertices = find_segments(active_vertices, segments)
    animation_vertices.append(np.array(active_vertices.copy()))
    animation_segments.append(np.array(visual.copy()))

    # active_vertices = np.vstack(active_vertices).T
    segments = np.vstack(np.array(segments)).T
    if visualization == 'end_picture':
        visual = np.array(visual).T
        return visual,active_vertices
    elif visualization == 'animation':
        return [animation_segments,animation_vertices]

In [16]:
animation_segments,animation_vertices = run(N,M,visualization='animation')
# visual,active_ver = run(N,M,visualization='end_picture')

/home/paulina/.anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:1273: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])
/home/paulina/.anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in less
  app.launch_new_instance()


In [17]:
animated_frames(animation_segments, animation_vertices)

FigureCanvasNbAgg()

IntSlider(value=0, max=876)

interactive(children=(Play(value=0, description='Press play', interval=500, max=876), Output()), _dom_classes=…

<function __main__.animated_frames.<locals>.update(f:int=0)>

In [27]:
# animation(animation_segments,animation_vertices,"anim")
# for i in animation_segments:
#     print(len(i))
# plt.plot(animation_vertices[0].T[0],animation_vertices.T[1],'o')
# plt.plot(animation_segments[0][0].T[0],animation_segments[0][0].T[1])
# animation_vertices
animation_segments[-1]

array([[[ 0.        ,  0.        ],
        [-0.00990527, -0.00137321]],

       [[-0.00990527, -0.00137321],
        [-0.01981053, -0.00274641]],

       [[-0.01981053, -0.00274641],
        [-0.0297158 , -0.00411962]],

       ...,

       [[-0.9328053 , -0.72791183],
        [-0.93609756, -0.71846933]],

       [[-0.93609756, -0.71846933],
        [-0.93938983, -0.70902682]],

       [[-0.93938983, -0.70902682],
        [-0.9426821 , -0.69958431]]])

In [ ]:
fig = plt.figure(figsize=[15,15])
# plt.plot(segments[0],segments[1],'.')
plt.plot(visual[0],visual[1])
if active_vertices != []:
    plt.plot(active_vertices[0],active_vertices[1],'o')
# plt.savefig("takze_tego1.png")

In [ ]:
fig = plt.figure(figsize=[10,10])
vis = np.swapaxes(animation_segments[-10],0,2)
# plt.plot(visual[0],visual[1])
plt.plot(vis[0],vis[1])
print(visual.shape)